In [100]:
from lxml import etree
import sqlite3

In [101]:
xml_db = 'wordnet/data/xml_synsets.db'

In [102]:
tree = etree.parse("estwn-et-2.0.0.beta.xml")

In [103]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except sqlite3.Error as e:
        print("Connection error: [%s]" % e)

    return None

def create_table(conn, create_table_sql ):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except sqlite3.Error as e:
        print("Connection error while creating table: [%s]" % e)

def sqlTables(databaseLoc):

    sql_create_synset_table = ''' CREATE TABLE IF NOT EXISTS xml_synsets(
                                        
                                        start_id TEXT NOT NULL,
                                        target_id TEXT NOT NULL,
                                        relation TEXT NOT NULL,
                                        status TEXT NOT NULL,
                                        confidenceScore INT NOT NULL
                                                    ); '''
    conn = create_connection(databaseLoc)
    if conn is not None:
        create_table(conn,sql_create_synset_table)
    else:
        print("Error! cannot create db conn.")

In [104]:

def fetch_data(tree):
    '''
    Upload .attrib['id'] - end_id
              [i].['target'] - start_id
              [i].['reltype'] - relation
              [0].['sourceSense'] - writtenForm
              [i].['confidenceScore']
              [i].['status']
    '''
    synsets = tree.xpath('/LexicalResource/Lexicon/Synset')
    j=0
    for ssets in synsets:
        end_estwn = ssets.attrib['id']
        if len(sset)>1:
            for sset in ssets:
                start_estwn = sset.attrib['target']               
                relation    = sset.attrib['relation']
                conf        = sset.attrib['confidenceScore']
                status      = sset.attrib['status']
        #TODO: else statement        
        else:
            try:'''
                #print("try @", j)
                word = sset[0].attrib['writtenForm']
                if " " in word:
                    var = word.replace(" ", "_")
                    word = var
                pos  = sset[0].attrib['partOfSpeech']
                string = "s-" + word + "-" + pos
                sset_list.append(string)
                estwnId_list.append(sset.attrib['id'])
            except:
                #print("pass @",j)
                pass
                '''
        j+=1
        '''
        for relations
        
        if len(sset) > 2:
            for data in sset:
                #print(data.attrib)
                
                status_list.append(data.attrib['target'])
                conf_list.append((data.attrib['confidenceScore']))
                source_sense.append(data[0].attrib['sourceSense'])           
                '''

In [105]:
def upload_data(db_file):
    sqlTables(db_file)
    conn = create_connection(db_file)
    cursor = conn.cursor()
    
    with conn:
        i=0
        for i in range(len(graph_id)):
              
            
            cursor.execute("INSERT INTO XML_LexicalEntry(estwn_id,synset, lemma, pos) VALUES(?,?,?,?)"\
                                                        ,(estwnId_list[i],sset_list[i],lemma_list[i], pos_list[i], ))#, status))
            conn.commit()

In [98]:
estwnId_list = []
status_list=[]
pos_list = []
lemma_list = []
sset_list=[]

fetch_data(tree)

In [99]:
upload_data(xml_db)

KeyboardInterrupt: 

In [107]:
synsets = tree.xpath('/LexicalResource/Lexicon/Synset')
print(len(synsets))

83557


In [114]:
sset = synsets[0]
print(len(sset))
print(sset[0].attrib)

106
{'language': 'et', 'sourceSense': 's-kalapüük-n1'}


The most important property of a synset is its identifier, status and confidence 

In [120]:
synset=synsets[0]
print('synset_id=', synset.attrib['id'])
print('synset_status=', synset.attrib['status'])
print('confidence_score=', synset.attrib['confidenceScore'])


synset_id= estwn-et-8645-n
synset_status= checked
confidence_score= 1.0


The first element of the synset chhild is its definition that links back to lexicon element senses

In [121]:
print(synset[0].tag)
print('language = ', synset[0].attrib['language'])
print('lecicon_sense = ', synset[0].attrib['sourceSense'])

Definition
language =  et
lecicon_sense =  s-kalapüük-n1


Next elements are relations between synsets spacified through target and relation type

In [141]:
i=0
for sset in synsets:
    if len(sset) > 2:
        print(i)
    elif len(sset) < 2:
        print("< 2: ", i)
    i+=1

0
1
2
4
< 2:  9
10
11
13
15
25
26
31
32
41
53
56
57
60
61
62
64
65
67
69
70
71
72
73
74
77
78
81
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
104
106
108
109
110
112
113
114
115
116
117
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
159
160
161
162
163
164
166
167
168
169
171
172
173
174
175
176
177
178
179
180
181
185
186
187
< 2:  189
191
192
193
196
197
198
199
200
201
205
206
207
208
209
210
211
212
214
220
221
222
226
227
229
231
243
248
256
262
270
271
272
285
293
294
295
300
302
305
306
307
310
312
313
314
317
318
319
320
321
322
324
325
328
330
332
335
336
340
342
343
345
346
354
356
357
358
359
360
362
363
< 2:  364
365
366
367
< 2:  368
< 2:  369
370
371
372
373
376
379
380
386
387
390
394
399
400
401
403
404
406
411
418
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
439
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
461
462
463


5523
5531
5532
5533
5548
5569
5587
5592
5610
< 2:  5611
5612
5613
5614
< 2:  5615
5616
5617
5618
5619
5620
< 2:  5622
< 2:  5624
< 2:  5625
5626
< 2:  5628
< 2:  5629
5631
5633
5634
5635
< 2:  5636
5639
5643
< 2:  5644
5645
5647
5650
5652
5653
5655
5657
5659
5660
5662
5663
5664
5665
5669
< 2:  5670
< 2:  5672
5673
5674
< 2:  5677
< 2:  5678
5679
< 2:  5680
5682
5683
5684
< 2:  5685
< 2:  5686
5688
5689
5690
5691
5693
5694
< 2:  5695
5696
5697
5698
5700
5701
5703
5704
< 2:  5705
5707
5708
5709
5712
< 2:  5713
< 2:  5714
5715
< 2:  5716
< 2:  5717
< 2:  5718
< 2:  5719
5720
< 2:  5721
5723
5724
5725
< 2:  5726
< 2:  5728
5730
< 2:  5731
< 2:  5732
< 2:  5733
< 2:  5734
5735
5736
< 2:  5737
< 2:  5738
< 2:  5739
< 2:  5740
< 2:  5742
5744
5745
5746
5747
5748
5749
5750
5751
5752
5753
5754
5755
5757
5760
5761
5762
5765
5769
5772
5782
5783
5789
5794
5796
5799
5807
5837
< 2:  5844
< 2:  5845
< 2:  5846
5849
< 2:  5851
< 2:  5852
5853
< 2:  5855
< 2:  5857
< 2:  5858
< 2:  5859
< 2:  5860
< 2:

9518
9519
9521
9522
9523
9525
9530
9531
< 2:  9532
9533
9534
9536
9538
9540
9541
9542
9545
9546
9547
9548
9549
< 2:  9551
9553
9557
9558
9561
9562
9563
< 2:  9564
9566
9567
9568
9570
9571
9573
9574
9575
9576
9582
9583
9584
9585
9586
9587
9588
9589
9591
9592
9593
9594
9595
9599
9600
9601
9602
9603
9605
9608
9609
9610
9611
9614
9615
9616
9617
9618
9621
9622
9623
9625
9626
9630
9631
9632
9633
9635
9636
9637
9638
9639
9640
9642
9643
9644
9645
9646
9648
9649
9651
9653
9655
9656
9657
9658
9660
9661
9663
9664
< 2:  9665
9666
9669
9670
9671
9672
9674
9675
9677
9678
9679
9680
9682
9683
9684
9685
9686
9687
9689
9690
9692
9693
9694
9695
9696
9697
9698
9699
9701
9702
9706
9707
9708
9710
9713
9714
9715
9716
9717
9718
9719
9720
9721
9722
9723
9725
9726
9727
9728
9730
9731
9732
9733
9735
9736
9737
9738
9739
9740
9741
9742
9743
9744
9745
9746
9747
9748
9749
9750
9751
9752
9753
9754
9755
9761
9762
9770
9771
9772
9774
9775
9777
9781
9782
9783
9784
9786
9787
9789
9790
9791
9793
9794
9795
< 2:  9796
9797


11473
11474
11475
11476
11477
< 2:  11478
11479
11481
11483
11485
11488
11489
11492
11495
11498
11499
11500
11501
11504
11507
11508
11509
11510
11511
11512
< 2:  11514
< 2:  11515
< 2:  11516
< 2:  11517
< 2:  11518
< 2:  11519
< 2:  11520
< 2:  11523
< 2:  11524
< 2:  11525
< 2:  11526
< 2:  11527
< 2:  11528
< 2:  11529
< 2:  11530
< 2:  11531
< 2:  11532
< 2:  11533
< 2:  11534
11536
11537
11539
11541
11543
11544
11546
11550
< 2:  11551
11552
11554
< 2:  11555
11556
11558
11559
11560
11561
11562
< 2:  11563
11564
11566
< 2:  11567
11568
< 2:  11570
11571
< 2:  11572
11573
< 2:  11574
< 2:  11575
11576
< 2:  11577
11578
11579
< 2:  11580
11581
< 2:  11582
11583
< 2:  11584
11585
< 2:  11586
11587
11588
< 2:  11589
11590
11592
< 2:  11593
11594
11595
11596
11597
< 2:  11598
11600
< 2:  11601
11602
11603
11605
11607
11609
< 2:  11610
11611
11613
11614
11615
11617
11618
< 2:  11619
11620
< 2:  11621
11622
11624
< 2:  11625
11626
11627
< 2:  11628
11629
< 2:  11630
11632
< 2:  11633
1163

13256
< 2:  13257
13258
< 2:  13259
13260
< 2:  13261
13264
< 2:  13265
13266
13267
13268
13269
13270
< 2:  13271
13272
< 2:  13273
13274
13275
< 2:  13276
13277
< 2:  13278
13279
13281
13283
< 2:  13284
13285
< 2:  13286
13287
< 2:  13288
13289
13290
< 2:  13291
13292
< 2:  13293
13294
< 2:  13295
13296
13297
13298
< 2:  13299
13300
13301
13303
13304
13305
13307
13309
13312
13313
13319
13320
13321
13323
13324
13325
13326
13330
13331
< 2:  13332
13333
13336
< 2:  13337
13338
13339
13341
13342
13343
13344
13346
< 2:  13347
13348
< 2:  13349
13350
13352
< 2:  13353
< 2:  13358
13359
< 2:  13360
13361
13362
< 2:  13363
13365
< 2:  13366
13367
< 2:  13368
13369
< 2:  13370
13371
13372
13373
13374
< 2:  13375
< 2:  13377
13380
< 2:  13381
13382
< 2:  13383
13384
< 2:  13387
13388
< 2:  13389
13390
13392
13393
13394
< 2:  13395
< 2:  13397
< 2:  13398
13399
< 2:  13400
13401
13403
13404
13406
< 2:  13407
13408
< 2:  13409
13410
< 2:  13411
13412
< 2:  13413
13414
13416
< 2:  13417
13418
1342

...

66417
66437
66439
66441
< 2:  66447
66448
66453
66454
66455
66462
66464
66466
66468
66469
< 2:  66470
< 2:  66473
66477
66478
66485
66486
66487
66488
66489
66491
66492
66493
66494
66495
66496
66497
66499
66503
66506
66508
66509
66513
66514
66515
66516
66518
66519
66522
66523
66526
66527
66533
66535
66536
66537
66550
< 2:  66553
< 2:  66554
66555
66566
66567
66569
66573
66579
66590
66591
66592
66593
66594
66595
66596
66597
66598
66599
66600
66601
66602
66603
66604
66605
66608
66610
66611
66612
66613
66614
66615
66617
66620
66621
66622
66623
66624
66626
66627
66631
66637
66642
66653
66655
66659
66660
66661
66662
66663
< 2:  66667
< 2:  66669
< 2:  66670
< 2:  66671
< 2:  66672
< 2:  66673
< 2:  66674
< 2:  66675
66683
66687
66688
66689
66690
66691
66693
66694
66696
66698
66702
66703
66705
66706
66708
66711
66716
66726
66727
66728
66729
66731
66733
66734
66735
66740
66741
66742
66743
66744
66745
66746
66747
66749
66750
66751
66752
66754
66755
66756
66757
66758
66759
66762
66763
66765
6676

69943
< 2:  69949
69955
69964
69968
69970
69973
69977
69978
69979
69980
69983
69985
69987
69988
69989
69992
< 2:  69994
< 2:  69995
< 2:  69996
< 2:  69997
< 2:  69998
69999
70002
< 2:  70003
< 2:  70004
< 2:  70005
< 2:  70007
< 2:  70010
70014
70015
70016
70018
70019
70025
70027
70028
< 2:  70033
< 2:  70036
70039
70042
70043
70045
< 2:  70046
70047
< 2:  70048
70049
70055
70057
70061
70065
< 2:  70066
70067
70068
70076
70077
70078
70079
70080
70083
70086
70088
70090
70091
70092
70093
70094
70096
70098
70100
70101
70103
< 2:  70105
70107
70110
70111
70113
70114
70116
70117
70120
70123
70124
70125
70126
70127
70128
70130
70131
70132
70137
70138
70140
70143
70144
70148
70155
< 2:  70157
70158
70164
70165
70166
70167
70168
70169
70172
70173
70175
70176
70177
70178
70179
70185
70187
70195
70196
70198
70202
70203
70207
70210
70211
70212
70226
70227
70228
70229
70230
70232
70233
70235
70238
70243
70250
70251
70254
70255
70256
70257
70258
70259
70260
70262
70263
70264
70265
70266
70267
7026

73279
73280
< 2:  73281
< 2:  73282
< 2:  73283
73285
73291
73297
< 2:  73298
< 2:  73299
73306
73308
73317
73318
73325
73328
73329
73331
73332
73333
73335
73336
73337
73338
73339
< 2:  73341
73343
73345
< 2:  73346
73347
73350
73352
73353
73361
73364
73365
73366
73368
73374
73380
73381
73404
73407
73408
73430
73431
73432
73433
73435
73442
73449
73461
73465
73472
73473
73474
73477
73478
73481
73482
73483
73484
73497
73501
73503
73506
73507
73516
73517
73521
73522
73526
73527
73528
73530
73531
73532
73535
73536
73537
73538
73539
73540
73541
73543
73544
73545
73550
73551
73552
73553
73556
73558
73563
73572
73583
73585
73586
73588
73593
73598
73607
73618
73622
73641
73646
73655
73656
73674
73675
73680
73682
73686
73687
73688
73694
73714
73733
73746
73747
73748
73749
73750
73751
73752
73754
< 2:  73756
73757
73759
73761
73769
73770
73773
73778
73779
73781
73784
73796
73797
73798
< 2:  73800
< 2:  73801
< 2:  73803
73806
73809
73816
73825
73834
73835
< 2:  73837
73845
73848
73856
73862
7386

77034
77035
77036
77042
77045
77047
77050
77051
77053
77054
77055
77056
77057
77058
77059
77060
77062
77063
77064
77065
< 2:  77066
77068
< 2:  77069
< 2:  77070
77071
77073
77074
77075
< 2:  77076
< 2:  77077
< 2:  77078
77079
< 2:  77080
77082
< 2:  77084
< 2:  77087
77089
77090
77091
77092
77093
77094
< 2:  77095
77096
< 2:  77098
77100
< 2:  77101
< 2:  77102
< 2:  77103
< 2:  77104
77105
77106
77107
77108
77109
77110
77112
77113
77114
77117
77118
77120
77121
77122
77124
77125
77126
77127
77128
77129
77130
77132
77134
77135
77139
77140
77141
77143
77145
77146
77147
77148
77150
77151
77153
77154
77155
77156
77158
77159
77160
77161
77162
77164
77165
77169
77170
77171
77172
77173
77175
77176
77178
77179
77181
77182
77183
77185
77192
77196
77199
< 2:  77203
< 2:  77204
< 2:  77205
77206
< 2:  77210
< 2:  77211
< 2:  77214
< 2:  77215
< 2:  77216
77217
77218
< 2:  77220
< 2:  77221
< 2:  77223
< 2:  77224
77227
< 2:  77228
< 2:  77229
77230
77232
77234
< 2:  77236
< 2:  77237
77238
7724

80201
80204
80205
80208
80211
80212
80213
80214
80215
80219
80224
80226
80227
80229
80232
80235
80236
80238
80239
80240
80241
80244
80246
80247
80248
80249
80250
80260
80261
80265
80266
80268
80269
80271
80272
80273
80274
80281
80282
80283
80284
80285
80286
80287
80288
80289
80290
80291
80294
80295
80296
80297
80300
80301
80302
80304
80305
80306
80307
80308
80309
80310
80312
80318
80319
80320
80321
80324
80327
80330
80331
80333
80334
80336
80337
80338
80342
80343
80344
80355
< 2:  80357
80358
< 2:  80359
< 2:  80360
< 2:  80361
< 2:  80362
< 2:  80364
< 2:  80365
< 2:  80366
< 2:  80367
< 2:  80368
< 2:  80369
< 2:  80370
< 2:  80371
< 2:  80372
< 2:  80373
< 2:  80374
< 2:  80375
< 2:  80376
< 2:  80377
< 2:  80378
< 2:  80379
< 2:  80380
< 2:  80381
< 2:  80382
< 2:  80383
< 2:  80384
< 2:  80385
< 2:  80386
< 2:  80387
< 2:  80388
< 2:  80389
< 2:  80390
< 2:  80391
< 2:  80392
< 2:  80393
< 2:  80394
< 2:  80395
< 2:  80396
< 2:  80397
< 2:  80398
< 2:  80399
< 2:  80400
< 2:  8040

In [163]:
synset = synsets[10]
print(len(synset))
print(synset[0].tag)
print('rel_confidence=', synset.attrib['confidenceScore'])
print('rel_type=', synset[0].attrib['relType'])
print('rel_status=', synset[0].attrib['status'])
print('rel_target=', synset[0].attrib['target'])
print('synset_id=', synset.attrib['id'])
print('sourceSense = ', synset[0].attrib['sourceSense'])


4
Definition
rel_confidence= 0.5


KeyError: 'relType'

You can access these elements by specifying relative paths

In [160]:
definition = synset.xpath('Definition')
relations = synset.xpath('SynsetRelation')
print(len(definition),len(relations))

1 105


In [44]:
print(len(synsets))
print(len(entries))

NameError: name 'synsets' is not defined

In [90]:
print(entry[1].attrib)
print(synset[100].attrib)

{'synset': 'estwn-et-41417-n', 'id': 's-röntgen-n1', 'status': 'unchecked'}
{'target': 'estwn-et-71125-n', 'relType': 'hyponym', 'status': 'unchecked', 'confidenceScore': '1.0'}
